## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [1]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score



import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


### Data Exploration

In [2]:
#check missing values
train_data.isnull().sum()

ID           0
Promotion    0
purchase     0
V1           0
V2           0
V3           0
V4           0
V5           0
V6           0
V7           0
dtype: int64

In [3]:
#check statistics of training data
train_data.describe()

,ID,purchase,V1,V2,V3,V4,V5,V6,V7
count,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000
mean,62970.972413,0.012303,1.500662,29.973600,0.000190,1.679608,2.327643,2.502898,1.701694
std,36418.440539,0.110234,0.868234,5.010626,1.000485,0.466630,0.841167,1.117349,0.457517
min,1.000000,0.000000,0.000000,7.104007,-1.684550,1.000000,1.000000,1.000000,1.000000
25%,31467.250000,0.000000,1.000000,26.591501,-0.905350,1.000000,2.000000,2.000000,1.000000
50%,62827.500000,0.000000,2.000000,29.979744,-0.039572,2.000000,2.000000,3.000000,2.000000
75%,94438.750000,0.000000,2.000000,33.344593,0.826206,2.000000,3.000000,4.000000,2.000000
max,126184.000000,1.000000,3.000000,50.375913,1.691984,2.000000,4.000000,4.000000,2.000000


In [4]:
#check data type
train_data.dtypes

ID             int64
Promotion     object
purchase       int64
V1             int64
V2           float64
V3           float64
V4             int64
V5             int64
V6             int64
V7             int64
dtype: object

In [5]:
#check promotion "yes" and "no" proportion
pro_no_promo = train_data.query('Promotion == "No"').shape[0]/train_data.shape[0]
pro_yes_promo = train_data.query('Promotion == "Yes"').shape[0]/train_data.shape[0]
print("proportion of no promotion = ",pro_no_promo,"proportion of promotion = ",pro_yes_promo)

proportion of no promotion =  0.4988525327087326 proportion of promotion =  0.5011474672912675


In [6]:
#subset data into groups with promotion and not
promotion_yes = train_data.query('Promotion == "Yes"')
promotion_no = train_data.query('Promotion == "No"')

In [7]:
# subset promoted into purchased and not purchased
purchased_yes = promotion_yes.query('purchase == 1')
purchased_no = promotion_yes.query('purchase == 0')
purchase_yes = promotion_no.query('purchase == 1')
purchase_no = promotion_no.query('purchase == 0')
print("purchase ratio of promoted = ",purchased_yes.shape[0]/promotion_yes.shape[0])
print("not purchased ratio of promoted = ", purchased_no.shape[0]/promotion_yes.shape[0])
print("purchased ratio of not promoted = ", purchase_yes.shape[0]/promotion_no.shape[0])
print("not purchased ratio of not promoted = ",purchase_no.shape[0]/promotion_no.shape[0])

purchase ratio of promoted =  0.017019167217448776
not purchased ratio of promoted =  0.9829808327825512
purchased ratio of not promoted =  0.007564619397676073
not purchased ratio of not promoted =  0.992435380602324


In [8]:
#number of purchased and not in promoted 
#promotion_yes.groupby("purchase").count()
purchase_pro = promotion_yes[promotion_yes.purchase == 1].count()[0]
not_purchase_pro = promotion_yes[promotion_yes.purchase == 0].count()[0]
print(purchase_pro,not_purchase_pro)

721 41643


In [9]:
#number of purchased and not in not promoted 
#promotion_no.groupby("purchase").count()
purchase_no_pro = promotion_no[promotion_no.purchase == 1].count()[0]
not_purchase_no_pro = promotion_no[promotion_no.purchase == 0].count()[0]
print(purchase_no_pro,not_purchase_no_pro)

319 41851


In [10]:
#define Incremental Response Rate (IRR)
"""it's the ratio of the number of purchasers in the promotion group to the total number of customers in the
purchasers group (treatment) minus the ratio of the number of purchasers in the non-promotional group to the
total number of customers in the non-promotional group (control)."""

IRR = purchased_yes.shape[0]/promotion_yes.shape[0] - purchase_yes.shape[0]/promotion_no.shape[0]
print(IRR)

0.009454547819772702


In [11]:
total_pro = promotion_yes.shape[0]
total_no_pro = promotion_no.shape[0]
print(total_pro,total_no_pro)

42364 42170


In [12]:
print(purchase_pro,total_pro,purchase_no_pro)

721 42364 319


In [13]:
#define Net Incremental Revenue (NIR)
"""NIR is 10 times
the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent 
out, minus 10 times the number of purchasers who were not given the promotion.
𝑁𝐼𝑅=(10⋅𝑝𝑢𝑟𝑐ℎ𝑡𝑟𝑒𝑎𝑡−0.15⋅𝑐𝑢𝑠𝑡𝑡𝑟𝑒𝑎𝑡)−10⋅𝑝𝑢𝑟𝑐ℎ𝑐𝑡𝑟𝑙 """

NIR = 10*purchase_pro-0.15*total_pro-10*purchase_no_pro
print(NIR)

-2334.5999999999995


# Modeling

In [18]:
# selecting "purchase = 1" rows
df_purchase = df.query('purchase == 1')
df_purchase.head()

,ID,Promotion,purchase,V2,V3,V1_0,V1_1,V1_2,V1_3,V4_1,...,V5_1,V5_2,V5_3,V5_4,V6_1,V6_2,V6_3,V6_4,V7_1,V7_2
107,162,Yes,1,33.030562,-0.732194,0,1,0,0,0,...,1,0,0,0,1,0,0,0,1,0
328,503,Yes,1,31.760033,1.518828,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1
352,534,Yes,1,17.512709,1.259095,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,1
383,580,Yes,1,31.043735,-1.338239,0,1,0,0,0,...,1,0,0,0,0,1,0,0,1,0
423,639,No,1,34.901993,-0.905350,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,1


In [19]:
# selecting "purchase = 0" rows
df_not_purchase = df.query('purchase == 0')
df_not_purchase.head()

,ID,Promotion,purchase,V2,V3,V1_0,V1_1,V1_2,V1_3,V4_1,...,V5_1,V5_2,V5_3,V5_4,V6_1,V6_2,V6_3,V6_4,V7_1,V7_2
0,1,No,0,30.443518,-1.165083,0,0,1,0,1,...,1,0,0,0,0,0,1,0,0,1
1,3,No,0,32.159350,-0.645617,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,1
2,4,No,0,30.431659,0.133583,0,0,1,0,1,...,1,0,0,0,0,0,0,1,0,1
3,5,No,0,26.588914,-0.212728,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,1
4,8,Yes,0,28.044332,-0.385883,0,0,0,1,1,...,1,0,0,0,0,1,0,0,0,1


In [20]:
# Randomly sample 1040 rows from not purchased dataset
df_random = df_not_purchase.sample(n=1040)
df_random.head()


,ID,Promotion,purchase,V2,V3,V1_0,V1_1,V1_2,V1_3,V4_1,...,V5_1,V5_2,V5_3,V5_4,V6_1,V6_2,V6_3,V6_4,V7_1,V7_2
51439,76612,No,0,27.237156,-1.424817,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,1
83971,125343,No,0,37.662927,0.826206,0,1,0,0,1,...,0,1,0,0,1,0,0,0,0,1
48588,72307,No,0,25.939583,-0.039572,0,0,0,1,1,...,0,1,0,0,0,0,1,0,0,1
36437,54237,Yes,0,35.827921,-1.251661,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,1
29968,44513,No,0,20.529626,0.566472,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1


In [21]:
# new training dataset with half purchased and half not purchased
df_train = pd.concat([df_purchase, df_random], axis=0)
df_train.shape

(2080, 21)

In [23]:
# create training and testing vars
X = df_train.loc[:,'V2':] 
y = df_train['purchase']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1664, 18) (1664,)
(416, 18) (416,)


In [25]:
# logistic regression modeling on new training data df_train
log_mod = LogisticRegression()
log_mod.fit(X_train, y_train)
preds = log_mod.predict(X_test)
confusion_matrix(y_test, preds) 

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([[106,  97],
       [ 73, 140]], dtype=int64)

In [26]:
accuracy_score(y_test, preds)

0.5913461538461539

In [27]:
#transform 0/1 array to Yes/No array
my_map = {0: "No", 1: "Yes"}
promotion = np.vectorize(my_map.get)(preds)
promotion

array(['Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'No',
       'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes',
       'No', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No',
       'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No',
       'Yes', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'No',
       'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes',
       'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes',
       'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'No',
       'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'No',
       'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes',
       'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes',
       'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes',
       'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No',
       'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'No'

In [28]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    # transform categorical variables
    df_train = pd.get_dummies(data=df, columns=['V1','V4', 'V5','V6','V7'])
    
        
    #predict
    pred = log_mod.predict(df_train)
    
    # transform 0/1 array to N0/Yes 
    my_map = {0: "No", 1: "Yes"}
    promotion = np.vectorize(my_map.get)(pred)
    
    return promotion

In [29]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy)

Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0187.

Your nir with this strategy is 358.05.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.01865012976953461, 358.04999999999995)